## Step 0: Importing dependencies

In [28]:
# Dependencies
import pandas as pd
from scipy.io import loadmat
import glob
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM,Dropout,Dense 
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf


## Step 1: Set up dataset functions
Class has 3 elements
- An ```__init__``` function that sets up your class and all the necessary parameters.
- An ```__len__``` function that returns the size of your dataset.
- An ```__getitem__``` function that given an index within the limits of the size of the dataset returns the associated image and label in tensor form.

In [29]:
# folder names
dc_folders = ['Europe', 'Japan', 'USA', 'Upsample']

# shuffling data 
LENDATA = 36 + 6 + 4 + 1 # number of driving cycles = 47
np.random.seed(42)
p = np.random.permutation(int(LENDATA-1))

# Implement the dataset class
class DrivingCyclesDataset(Dataset):
    def __init__(self,
                 path_to_dc,
                 train=True):
        # path_to_dc: where you put the driving cycles dataset
        # train: return training set or test set
        
        # Load all the driving cycles
        alldata = []
        dcnames = []
        if (train == True):
            mat = loadmat('./DrivingCycles/WLTPextended.mat')
            df = pd.DataFrame(mat['V_z'], columns = ['V_z']) # velocity 
            df2 = pd.DataFrame(mat['T_z'], columns = ['T_z']) # time
            df3 = pd.DataFrame(mat['D_z'], columns = ['D_z']) # acceleration
            df = pd.concat([df, df2, df3], axis=1)
            alldata.append(df)
            dcnames.append('WLTPextended.mat')
            for folder in dc_folders:
                image_path = os.path.join(path_to_dc, folder)
                files = glob.glob(image_path + '/*.mat')
                for f in files:
                    mat = loadmat(f)
                    df = pd.DataFrame(mat['V_z'], columns = ['V_z'])
                    df2 = pd.DataFrame(mat['T_z'], columns = ['T_z'])
                    df3 = pd.DataFrame(mat['D_z'], columns = ['D_z'])
                    df = pd.concat([df, df2, df3], axis=1)
                    dcnames.append(os.path.basename(f))
                    # each dataframe is a driving cycle 
                    alldata.append(df)
            # Extract the driving cycles with the specified file indexes     
            self.data = (np.array(alldata, dtype=object))[p] #numpy array of dataframes 
            self.names = (np.array(dcnames, dtype=object))[p]
        
        else:
            image_path = os.path.join(path_to_dc, 'test')
            files = glob.glob(image_path + '/*.mat')
            for f in files:
                mat = loadmat(f)
                df = pd.DataFrame(mat['V_z'], columns = ['V_z'])
                df2 = pd.DataFrame(mat['T_z'], columns = ['T_z'])
                df3 = pd.DataFrame(mat['D_z'], columns = ['D_z'])
                df = pd.concat([df, df2, df3], axis=1)
                dcnames.append(os.path.basename(f))
                # each dataframe is a driving cycle 
                alldata.append(df)

            self.data = alldata
            self.names = dcnames


    def __len__(self, idx):
        # Return the number of samples in a driving cycle 
        return (self.data[idx]).size
        
    def __getitem__(self, idx):
        # Get an item using its index
        # Return the driving cycle and its name 
        return self.data[idx]

In [30]:
def create_dataset(dataset, h, f, test):
    x = [] #append the last h values
    y = [] #append the future f value 
    for df in dataset:
        features_considered = ['V_z', 'D_z']
        # features_considered = ['V_z']
        features = df[features_considered]
        for i in range(h, df.shape[0]-f):
            # for each driving cycle dataframe, have sets of (h+f) values 
            # h values are past values, f values are future value 
            features['v_ave'] = df['V_z'][i-h:i].mean()
            features['v_max'] = df['V_z'][i-h:i].max()
            features['v_min'] = df['V_z'][i-h:i].min()
            features['a_ave'] = df['D_z'][i-h:i].mean()
            features['a_max'] = df['D_z'][i-h:i].max()
            features['a_min'] = df['D_z'][i-h:i].min()
            x.append(features[i-h:i])
            if (test == False):
                y.append(df['V_z'][i:i+f])
            else:
                y.append(df['V_z'][i])
    x = np.array(x) 
    y = np.array(y)  
    return x,y

## Step 2: Exploring the dataset

### Step 2.1: Data visualisation.

In [ ]:
# loading datasets
dc_path = './DrivingCycles/'
dataset_train  = DrivingCyclesDataset(dc_path, train=True)
dataset_test = DrivingCyclesDataset(dc_path, train=False)

# Plot 1 sample from the test set
sample_data = dataset_test.data[0]
sample_name = dataset_test.names[0]
plt.title(sample_name + " Driving Cycle")
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.plot(sample_data['T_z'], sample_data['V_z'])
plt.show()

### Step 2.2 Load training and test datasets



In [ ]:
# scaling the datasets 
scaler = MinMaxScaler(feature_range=(0,1))
for df in dataset_train: 
    df['V_z'] = scaler.fit_transform(df[['V_z']])
    df['D_z'] = scaler.fit_transform(df[['D_z']])
for df in dataset_test: 
    df['V_z'] = scaler.fit_transform(df[['V_z']])
    df['D_z'] = scaler.fit_transform(df[['D_z']])

# parameters h and f
h = 15 # length of historical sequence
f = 5 # length of forecast sequence 

# create training set and test set 
pd.options.mode.chained_assignment = None
x_train, y_train = create_dataset(dataset_train, h, f, test=False)
x_test, y_test = create_dataset(dataset_test, h, f, test=True)

# check 
print(x_train.shape)
print(x_test.shape)

# reshaping input to LSTM model 
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

## Step 3: LSTM
In this section we will try to make a LSTM predictor to predict the future velocity. 

### Step 3.1: Hyperparameter Search 


In [ ]:
# importing libraries
import keras
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from keras.callbacks import EarlyStopping

# defining model space search
def build_model(hp):
    model = Sequential()
    # tuning number of neurons
    model.add(LSTM(hp.Int('first_units',min_value=32,max_value=256,step=32),return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2])))
    model.add(Dropout(hp.Float(f'Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    # tuning number of layers
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=256,step=32),return_sequences=True))
        model.add(Dropout(hp.Float(f'Dropout_{i}_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(LSTM(hp.Int('last_units',min_value=32,max_value=256,step=32)))
    # dropout layer
    model.add(Dropout(hp.Float('Dropout_last_rate',min_value=0,max_value=0.5,step=0.1)))
    # dense layer 
    model.add(Dense(y_train.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])
    return model

# create Tuner object
tuner= BayesianOptimization(
        build_model,
        objective='mse',
        max_trials=10,
        executions_per_trial=1,
        directory=os.path.normpath('./keras_tuning'),
        project_name='kerastuner_bayesian',
        overwrite=True
        )

# hyperparameter search 
early_stopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True)
tuner.search(x_train, y_train,epochs=5, steps_per_epoch = 200,
     validation_split=0.2,verbose=1, callbacks=[early_stopping])

# best model
best_model = tuner.get_best_models(num_models=1)[0]
print(best_model.summary())

# best hyperparameter
best_hyperparameters = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print('HyperParameters: {}'.format(best_hyperparameters))

# prediction 
predictions=best_model.predict(x_test)

# saving model 
best_model.save('bestLSTM.h5')

#### old model

In [33]:
def build_model(): 
    model = Sequential()
    model.add(LSTM(units=32, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2]))) # units depends on create_dataset function
    model.add(Dropout(0.2))
    model.add(LSTM(units=256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=256))
    model.add(Dropout(0.2))
    model.add(Dense(units=f))
    return model

### Step 3.2: Load and compile model 

In [ ]:
model = load_model('bestLSTM.h5') 

model.compile(loss='mse', optimizer='adam')

# check
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(model.summary())

### Step 3.2: Train the model.

In [ ]:
# early stopping 
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True)

# model checkpoint 
model_checkpoint = ModelCheckpoint(filepath="mymodel_{epoch}.h6", save_best_only=True, monitor='val_loss', verbose=1)

# Prepare a directory to store all the checkpoints.
checkpoint_dir = "./ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    model = load_model('bestLSTM.h5') 
    model.compile(loss='mse', optimizer='adam')
    return model

model = make_or_restore_model()

# start training
model.fit(x_train, y_train, epochs=10, batch_size=50, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])
model.save('speed_prediction.h5')

# load the model 
model = load_model('speed_prediction.h5') 

In [ ]:
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2]))) # units depends on create_dataset function
model.add(Dropout(0.2))
model.add(LSTM(units=256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=256))
model.add(Dropout(0.2))
model.add(Dense(units=f))

# compile the model 
model.compile(loss='mse', optimizer='adam')
# early stopping 
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True)
model.fit(x_train, y_train, epochs=10, batch_size=50, validation_split=0.2, callbacks=[early_stopping])
model.save('speed_prediction.h5')

# load the model 
model = load_model('speed_prediction.h5') 

### Step 3.3: Deploy the trained model onto the test set. 

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# check
print(predictions.shape)

In [ ]:
y_test_scaled = scaler.inverse_transform(y_test.reshape(-1,1))

#check
print(y_test_scaled.shape)

### Step 3.4: Evaluate model performance

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=50)
print("Min-max scaled MSE loss: ", results)

# other metrics
def evaluate_unscaled (predictions, y_test_scaled): 
    mse_ = keras.losses.MeanSquaredError()

    mse = mse_(predictions,y_test_scaled)
    print('Transformed MSE loss:', mse.numpy())

evaluate_unscaled(predictions, y_test_scaled)

### Step 3.5 Time lag

In [ ]:
time_shift = 0 
predictions_df = pd.DataFrame(predictions)

# introduce lag time in predictions
for i in range(5):
    predictions_shifted = predictions_df.shift(i)
    mae_ = keras.losses.MeanAbsoluteError()
    time_shift = min(time_shift, mae_(predictions_shifted.to_numpy(),y_test_scaled))
print(time_shift)

### Step 3.6: Perturbation of input driving cycle parameters

In [ ]:
# Perturbation 
def var_importance(model):
    for i in range(x_test.shape[2]):  # iterate over the three features
        new_x = x_test.copy()
        perturbation = np.random.normal(0.0, 0.5, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = model.predict(new_x)
        effect = ((predictions - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable {i+1}, perturbation effect: {effect:.4f}')

var_importance(model)

### Step 3.7: Visualisation of prediction

In [ ]:
# Combination information
combi = 'S6'

# plotting
fig, ax = plt.subplots(figsize=(16,4))
ax.plot(y_test_scaled)
# plt.plot(predictions, color='red') 
for i in range(predictions.shape[0]): 
    plt.plot(range(i,i+f), predictions[i], color='red') #uncomment for all forecast sequences
plt.legend(['Actual velocity', 'Predicted velocity'])
title = 'Predicted future velocity profile for h=' + str(h) + ' and f=' + str(f) + ' for combination ' + str(combi)
plt.title(title)
plt.xlabel("Velocity (m/s)")
plt.ylabel("Time (s)")

## Step 4: Baseline model


In [ ]:
# Baseline model 
# assumption that the forecast is equal to the mean of the historical time sequence 
def create_baseline_results(dataset, h, f):
    y = []  
    y_true = []
    for df in dataset:
        for i in range(h, df.shape[0]-f):
            # for each driving cycle dataframe, have sets of 51 values 
            # h values are past values, f values are future value 
            features = df['V_z']
            features['V_z'] = df['V_z'][i-h:i].mean()
            y.append(features[i-h:i])
            y_true.append(df['V_z'][i:i+f])
    y = np.array(y)  
    y_true = np.array(y_true) 
    return y, y_true

dc_path = './DrivingCycles/'
dataset_test = DrivingCyclesDataset(dc_path, train=False)

# parameters h and f
h = 10 # length of historical sequence
f = 10 # length of forecast sequence 

# create training set and test set 
pd.options.mode.chained_assignment = None
predictions, y_hat = create_baseline_results(dataset_test, h, f)

# evaluating the model
print(predictions.shape)
print(y_hat.shape)
mse_ = tf.keras.losses.MeanSquaredError()
mse = mse_(predictions,y_hat)
print('mse:', mse)